In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import time
import datetime
import os
from bs4 import BeautifulSoup as bs

In [2]:
#driver = webdriver.Chrome(r"C:\Users\chpar10\OneDrive - IKEA\DA\chromedriver.exe")
driver = webdriver.Chrome(r"D:\Git\chromedriver.exe")

In [3]:
driver.get('http://kyochon.revelup.com')



In [5]:
revel_id = "gyutae"
driver.find_element_by_xpath('//*[@id="auth0-lock-container-1"]/div/div[2]/form/div/div/div/div[2]/div[2]/span/div/div/div/div/div/div/div/div/div/div/div[1]/div/input').send_keys(revel_id)
print()
passwd = "Teddygom850212!"
print()
driver.find_element_by_xpath('//*[@id="auth0-lock-container-1"]/div/div[2]/form/div/div/div/div[2]/div[2]/span/div/div/div/div/div/div/div/div/div/div/div[2]/div[1]/div/input').send_keys(passwd)
driver.find_element_by_xpath('//*[@id="auth0-lock-container-1"]/div/div[2]/form/div/div/div/button').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="navigation"]/div/ul/li[2]/a/span').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
time.sleep(2)

In [7]:
# sales 추출 관련 function
def sto_summary():
    # Stores
    sto = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[5]/div[2]/span').text
    now = datetime.datetime.now()
    # Dates
    now_date = str(now.year) + '-' + ('0' + str(now.month))[-2:] + '-' + ('0' + str(now.day))[-2:]
    # Time
    now_hour = str(now.hour)
    # Net sales
    acc_sales = driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]//ul/li[1]/div/div[2]').text
    # Net transaction
    acc_transaction = driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[2]/ul/li[1]/ul/li[1]/div/div[2]').text
    # avg check (calculated)]
    try:
        avg_check = float(acc_sales)/float(acc_transaction)
    except ZeroDivisionError:
        avg_check = 0
        
    table1 = pd.DataFrame([sto, now_date, now_hour, acc_sales, acc_transaction, avg_check], 
                          index = ['Store', 'Date', 'Hour', 'Sales', 'Transaction','avg check'],
                          columns = ['Summary'])
    table1 = table1.reset_index()
    return table1

In [8]:
# Range Mix 추출 관련 function
def prod_mix():
    tbl_len = len(driver.find_elements_by_class_name('last-matrix-element'))
    table2 = {}
    for i in range(tbl_len):
        table2[driver.find_elements_by_class_name('product-description-container')[i].text] = {'Qty': driver.find_elements_by_class_name('n_items')[i+1].text, 
                                                                                              'Sales': driver.find_elements_by_class_name('price')[i+1].text}
    table2 = pd.DataFrame(table2).T.reset_index()
    table2 = table2.iloc[:len(table2)-2]
    return table2

In [9]:
# 스토어 별로 Loop 돌면서 함수 실행
driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[5]/div[2]/span').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="establishments-tree"]/div/div[2]/div[2]/div[2]/span[1]').click()
n_sto = len(driver.find_elements_by_class_name('fancytree-title.disabled'))

summ = []
range_mix = []

for i in range(n_sto):
    # 중간의 time.sleep() 함수는 필요에 따라 조정 가능
    # 한국에서 테스트 시(접속 시) 페이지 로딩까지 시간이 걸려 에러가 발생함을 확인하여 여유롭게 sleep time 지정
    if i!=0:
        driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[5]/div[2]/span').click()
        time.sleep(5)
        driver.find_element_by_xpath('//*[@id="establishments-tree"]/div/div[2]/div[2]/div[2]/span[1]').click()
    time.sleep(5)
    driver.find_elements_by_class_name('fancytree-title.disabled')[i].click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="navigation"]/div/ul/li[2]/a/span').click()
    time.sleep(5)
    summ.append(sto_summary())
    driver.find_element_by_xpath('//*[@id="report_sub_nav"]/li[4]/a').click()
    time.sleep(5)
    range_mix.append(prod_mix())
    
    print('{} finished'.format(driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[5]/div[2]/span').text))

Element Fresh (Pavillion) finished
Midvalley Southkey finished
R&F Mall finished
TOPPEN finished


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]//ul/li[1]/div/div[2]"}
  (Session info: chrome=84.0.4147.135)


In [11]:
summ[0]

,index,Summary
0,Store,Element Fresh (Pavillion)
1,Date,2020-08-26
2,Hour,23
3,Sales,3288.68
4,Transaction,33
5,avg check,99.657


In [26]:

from openpyxl import Workbook


In [12]:
sales_summ = pd.DataFrame()

In [16]:
pd.concat([sales_summ, pd.DataFrame([summ[0].iloc[0,1],summ[0].iloc[3,1]]).T])

,0,1
0,Element Fresh (Pavillion),3288.68


In [19]:
for i in range(len(summ)):
    sales_summ = pd.concat([sales_summ, pd.DataFrame([summ[i].iloc[0,1],summ[i].iloc[3,1]]).T])

In [20]:
sales_summ

,0,1
0,Element Fresh (Pavillion),3288.68
0,Midvalley Southkey,5096.19
0,R&F Mall,0.00
0,TOPPEN,3468.42


In [ ]:
wb = Workbook()
for i in range(len(summ)):
    ws = wb.create_sheet()
    nrow, ncol = summ[i].shape
    for r in range(nrow):
        for c in range(ncol):
            ws.cell(r+1,c+1, value = summ[1].iloc[r,c])
    ws.title = ws.cell(1,2).value
    
    nrow, ncol = range_mix[i].shape
    for r in range(nrow):
        for c in range(ncol):
            ws.cell(r+1,c+4, value = range_mix[i].iloc[r,c])
wb.remove(wb['Sheet'])

now = datetime.datetime.now()
wb.save("Summary {y}{m}{d}.xlsx".format(y=now.year, m=('0' + str(now.month))[-2:], d=('0' + str(now.day))[-2:]))

In [93]:
nrow, ncol = summ[0].shape

In [94]:
for r in range(nrow):
    for c in range(ncol):
        print(r,c)
        ws.cell(r+1,c+1, value = summ[1].iloc[r,c])

0 0
0 1
1 0
1 1
2 0
2 1
3 0
3 1
4 0
4 1
5 0
5 1


In [95]:
ws.title = ws.cell(1,2).value

In [96]:
nrow, ncol = range_mix[1].shape

In [97]:
for r in range(nrow):
    for c in range(ncol):
        print(r,c)
        ws.cell(r+1,c+4, value = range_mix[1].iloc[r,c])

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2
5 0
5 1
5 2
6 0
6 1
6 2
7 0
7 1
7 2
8 0
8 1
8 2
9 0
9 1
9 2
10 0
10 1
10 2
11 0
11 1
11 2
12 0
12 1
12 2
13 0
13 1
13 2
14 0
14 1
14 2
15 0
15 1
15 2


In [103]:
wb.remove(wb['Sheet'])

In [104]:
now = datetime.datetime.now()
wb.save("Summary {y}{m}{d}.xlsx".format(y=now.year, m=('0' + str(now.month))[-2:], d=('0' + str(now.day))[-2:]))

In [62]:
os.getcwd()

'd:\\Git\\Projects\\Project_KC\\Crawling (2020 Jul)'

In [34]:
ws.title = summ[0].iloc[0,0]

In [ ]:
ws

In [65]:
range_mix[1].reset_index().iloc[:len(range_mix[1])-2]

,index,Qty,Sales
0,Ala Carte,2.00,41.50
1,Bottled Water,4.00,15.11
2,Combo,28.00,-0.02
3,Crispy Series,3.00,83.01
4,Honey,19.00,494.29
5,Hot / Cold Teas,7.00,42.47
6,Mixed,25.00,834.00
7,Original / Red,9.00,186.79
8,Packaging,5.00,8.49
9,Red Pepper,15.00,366.99


In [18]:
summ[0]

,Summary
Store,Element Fresh (Pavillion)
Date,2020-08-11
Hour,21
Sales,1588.21
Transaction,21
Avg. check,75.629


In [32]:
//*[@id="establishments-tree"]/div/div[3]/ul/li[1]/ul/li[2]/ul/li[6]/ul/li[1]/ul/li[2]/span[5]/span[3]

[                               Summary
 Store        Element Fresh (Pavillion)
 Date                        2020-07-31
 Hour                                16
 Sales                          1451.21
 Transaction                         17
 Avg. check                     85.3653,
                         Summary
 Store        Midvalley Southkey
 Date                 2020-07-31
 Hour                         16
 Sales                   1645.01
 Transaction                  26
 Avg. check              63.2696,
                 Summary
 Store          R&F Mall
 Date         2020-07-31
 Hour                 16
 Sales            516.03
 Transaction           9
 Avg. check      57.3367,
                 Summary
 Store            TOPPEN
 Date         2020-07-31
 Hour                 16
 Sales           1532.03
 Transaction          30
 Avg. check      51.0677,
                 Summary
 Store           Arcoris
 Date         2020-07-31
 Hour                 16
 Sales           1937.74
 Transacti

In [44]:
pd.concat(summ, axis=1)

,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary,Summary
Store,Element Fresh (Pavillion),Midvalley Southkey,R&F Mall,TOPPEN,Arcoris,MyTown,Pavilion,Pearl,The Gardens Mall,Wisma Central (D.K),AEON Nilai,Genting,IOI City Mall,1 Utama
Date,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31,2020-07-31
Hour,16,16,16,16,16,16,16,16,16,16,16,16,16,16
Sales,1549.12,1783.85,516.03,1551.84,2026.42,7033.15,3657.77,3711.89,5143.03,371.71,2351.45,4266.72,3320.12,8087.48
Transaction,18,29,9,31,33,111,63,63,89,7,35,53,49,121
Avg. check,86.0622,61.5121,57.3367,50.0594,61.4067,63.3617,58.0598,58.9189,57.7869,53.1014,67.1843,80.5042,67.7576,66.8387


In [45]:
i

22

In [4]:
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]/div/div[1]').click()
driver.find_element_by_xpath('//*[@id="table_data_place"]/div[1]/div[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]/ul/li[1]/div/div[2]').text

'116'